### ATTENTION - Restatement of flow and code is under progress. 

####### little massy right now

In [0]:
from poirot.azure.databricks import svc_prncpl_magic_setup
svc_prncpl_magic_setup(scope="kv-8451-tm-media-dev", app_id_secret="spTmMediaDev-app-id", app_pw_secret="spTmMediaDev-pw")


In [0]:
#Define service principals
service_credential = dbutils.secrets.get(scope='kv-8451-tm-media-dev',key='spTmMediaDev-pw')
service_application_id = dbutils.secrets.get(scope='kv-8451-tm-media-dev',key='spTmMediaDev-app-id')
directory_id = "5f9dc6bd-f38a-454a-864c-c803691193c5"
storage_account = 'sa8451dbxadhocprd'

In [0]:
spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net", service_application_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net", service_credential)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net", f"https://login.microsoftonline.com/{directory_id}/oauth2/token")

In [0]:
#Importing All the libraries
import pandas as pd
import numpy as np
import kayday as kd
import seg
import toolbox.config as con
import pyspark.sql.functions as f
import pyspark.sql.types as t
import datetime as dt
import matplotlib.pyplot as plt
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from functools import reduce
from datetime import datetime, timedelta
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.types import *
from effodata import ACDS, golden_rules, Sifter, Equality, Joiner
from toolbox.config import segmentation
from kpi_metrics import KPI, AliasMetric, CustomMetric, AliasGroupby, available_metrics, get_metrics
from seg.utils import DateType
from flowcate.files import FilePath
from pyspark.sql.functions import col, udf
from pyspark.sql.types import BooleanType
from pyspark.dbutils import DBUtils



In [0]:

timestamp = "abfss://media@sa8451dbxadhocprd.dfs.core.windows.net/audience_factory/egress_receipts/"
eligibility_path = ('abfss://landingzone@sa8451entlakegrnprd.dfs.core.windows.net/mart/comms/prd/fact/eligibility_fact')
files_path = dbutils.fs.ls(eligibility_path)

sorted_files = sorted(files_path, key=lambda x: x[1], reverse=True)

# Get the second latest file
second_latest_file = sorted_files[1]

print("The second latest file is: ", second_latest_file.path)

latest_eligibility_df = spark.read.parquet(eligibility_path) #This will give us the Dates we are using in. #Getting Eligiblity Table


### Getting all the Segmentation

In [0]:
def get_all_segmentation_name():
    '''
    Get all the Segmentation names
    
    '''
    segmentation_names = con.segmentations.all_segmentations

    return segmentation_names

Combining All Segmentations Together

In [0]:
from pyspark.dbutils import DBUtils

dbutils = DBUtils()
dir_contents = dbutils.fs.ls(timestamp)
dir_contents = [ x[0] for x in dir_contents]
segmentation_name = get_all_segmentation_name()

In [0]:
# Function to check if any column contains 'Y'
def any_column_contains_Y(*args):
    return 'Y' in args

In [0]:
my_schema = t.StructType([
    t.StructField("EHHN", t.StringType(), True),
    t.StructField("SEGMENT", t.StringType(), True),
    t.StructField("SEGMENTATION", t.StringType(), True),
    t.StructField("FRONTEND_NAME", t.StringType(), True),
    t.StructField("PROPENSITY", t.StringType(), True),
    t.StructField("SEGMENT_TYPE", t.StringType(), True),
    t.StructField("PERCENTILE_SEGMENT", t.StringType(), True),
    t.StructField("PERIOD", t.StringType(), True),
    t.StructField("TAGS", t.StringType(), True)
])
df = spark.createDataFrame([], schema=my_schema)
my_schema_elig = t.StructType([
    t.StructField("EHHN", t.StringType(), True),
    t.StructField("SEGMENT", t.StringType(), True),
    t.StructField("SEGMENTATION", t.StringType(), True),
    t.StructField("FRONTEND_NAME", t.StringType(), True),
    t.StructField("PROPENSITY", t.StringType(), True),
    t.StructField("SEGMENT_TYPE", t.StringType(), True),
    t.StructField("PERCENTILE_SEGMENT", t.StringType(), True),
    t.StructField("PERIOD", t.StringType(), True),
    t.StructField("TAGS", t.StringType(), True),
    t.StructField("ONSITE_COUNT", t.IntegerType(), True),
    t.StructField("OFFSITE_COUNT", t.IntegerType(), True),
    t.StructField("OVERALL_ELIGIBILITY", t.BooleanType(), True),
])
combined_segmentation_df = spark.createDataFrame([], schema=my_schema_elig)
leftover_segment = {}
for i in dir_contents:
    for segmentations in segmentation_name:
          try:
            segment = con.segmentation(segmentations)
            frontend_name = segment.frontend_name
            propensities = segment.propensities
            final_propensity = "".join(propensities)
            tag = segment.tags
            final_tag = "".join(tag)
            type_of_segment = segment.segment_type
            percentile_of_segment = segment.type
            latest_file = i
            rec = spark.read.csv(latest_file, header=True)
            path = rec.filter(f.col('audience')== segmentations).select('filepath').distinct().collect()[0][0]
            period = latest_file[-12:-4]
            segment_file = spark.read.format("delta").load(path)

            segment_file = (
                  segment_file.withColumn("SEGMENTATION", f.lit(segmentations))
                  .withColumn('FRONTEND_NAME', f.lit(frontend_name))
                  .withColumn("PROPENSITY", f.lit(final_propensity))
                  .withColumn("SEGMENT_TYPE", f.lit(type_of_segment))
                  .withColumn("PERCENTILE_SEGMENT", f.lit(percentile_of_segment))
                  .withColumn("PERIOD", f.lit(period))
                  .withColumn("TAGS", f.lit(final_tag))
            
            )
            
            segment_file = segment_file.filter(f.col("SEGMENT").isin(segment.propensities))
            segment_file = segment_file.select("EHHN", "SEGMENT", "SEGMENTATION", "FRONTEND_NAME",
                                                "PROPENSITY", "SEGMENT_TYPE", "PERCENTILE_SEGMENT","PERIOD","TAGS")
            df = df.union(segment_file)
            eligibility_analytical_dataset = df.join(latest_eligibility_df, on="ehhn", how='left')

            ### overall logic:

            column_names = ['NATIVE_ELIGIBLE_FLAG','TDC_ELIGIBLE_FLAG','SSE_ELIGIBLE_FLAG','FFD_ELIGIBLE_FLAG','EMAIL_ELIGIBLE_FLAG','PUSH_FLAG',
                            'FACEBOOK_FLAG','PANDORA_FLAG','CHICORY_FLAG','PUSH_FLAG','PREROLL_VIDEO_ELIGIBLE_FLAG','PINTEREST_ELIGIBLE_FLAG','ROKU_FLAG']

            # Register UDF
            contains_Y_udf = udf(any_column_contains_Y, BooleanType())

            # Apply UDF to create a new column


            eligibility_with_on_off_overall_flag = eligibility_analytical_dataset.withColumn('ONSITE_COUNT', f.when(((f.col('TDC_ELIGIBLE_FLAG') == 'Y') | (f.col('SSE_ELIGIBLE_FLAG') == 'Y') | (f.col('FFD_ELIGIBLE_FLAG') == 'Y') | (f.col('SS_ELIGIBLE_FLAG') == 'Y') | (f.col('PUSH_FLAG') == 'Y')), '1').otherwise('0').cast('integer'))\
                            .withColumn('OFFSITE_COUNT', f.when(((f.col('FACEBOOK_FLAG') == 'Y') | (f.col('LIVERAMP_FLAG') == 'Y') | (f.col('PANDORA_FLAG') == 'Y') | (f.col('PINTEREST_ELIGIBLE_FLAG') =='Y') | (f.col('PREROLL_VIDEO_ELIGIBLE_FLAG') == 'Y') | (f.col('CBA_ELIGIBLE_FLAG') == 'Y')), '1').otherwise('0').cast('integer'))\
                            .withColumn("OVERALL_ELIGIBILITY", contains_Y_udf(*[col(column) for column in column_names]))

            secondary_page_table = (eligibility_with_on_off_overall_flag.select("EHHN","SEGMENT","SEGMENTATION","FRONTEND_NAME","PROPENSITY","SEGMENT_TYPE","PERCENTILE_SEGMENT","PERIOD","TAGS","ONSITE_COUNT","OFFSITE_COUNT","OVERALL_ELIGIBILITY"))
                            
          
          except Exception as e:
                leftover_segment[segmentations] = (str(e) + str(dir_contents))
                pass

    combined_segmentation_df = combined_segmentation_df.union(secondary_page_table) #Get all segments combined
    



In [0]:
leftover_segment

In [0]:
combined_segmentation_df.cache()

In [0]:
# print(combined_segmentation_df.show(25, truncate=False))

In [0]:
segmentation_count_df = (combined_segmentation_df.groupBy('SEGMENTATION', 'PERIOD')
                        .agg(f.count('EHHN').alias("TOTAL_HH"),
                        f.max("frontend_name").alias("FRONTEND_NAME"),
                        f.max("SEGMENT").alias("SEGMENT"),
                        f.max("TAGS").alias("TAGS"),
                        f.max("PROPENSITY").alias("PROPENSITY"),
                        f.max("SEGMENT_TYPE").alias("SEGMENT_TYPE"),
                        f.max("PERCENTILE_SEGMENT").alias("PERCENTILE_SEGMENT"),
                        )
                        .select("SEGMENTATION","PERIOD","TOTAL_HH","FRONTEND_NAME","SEGMENT","TAGS","PROPENSITY","SEGMENT_TYPE","PERCENTILE_SEGMENT")
                        
                  )
                  
segmentation_onsite_offsite_count = (combined_segmentation_df.groupBy('SEGMENTATION', 'PERIOD')
                        .agg(f.count('EHHN').alias("TOTAL_HH"),
                        f.max("frontend_name").alias("FRONTEND_NAME"),
                        f.max("SEGMENT").alias("SEGMENT"),
                        f.max("PROPENSITY").alias("PROPENSITY"),
                        f.max("SEGMENT_TYPE").alias("SEGMENT_TYPE"),
                        f.max("PERCENTILE_SEGMENT").alias("PERCENTILE_SEGMENT"),
                        f.sum('ONSITE_COUNT').alias('ONSITE_COUNTS'),
                        f.sum('OFFSITE_COUNT').alias('OFFSITE_COUNTS'),
                        f.max("TAGS").alias("TAGS")
                        )
                        .select("SEGMENTATION", "PERIOD", "TOTAL_HH", "FRONTEND_NAME", "SEGMENT", "PROPENSITY", "SEGMENT_TYPE", "PERCENTILE_SEGMENT", "ONSITE_COUNTS", "OFFSITE_COUNTS", "TAGS")
                        
                    )

In [0]:
segmentation_count_df.coalesce(1).write.mode("overwrite").parquet(f"abfss://media@sa8451dbxadhocprd.dfs.core.windows.net/audience_factory/powerbi_inputs/household_counts/")

In [0]:
segmentation_onsite_offsite_count.coalesce(1).write.mode("overwrite").parquet(f"abfss://media@sa8451dbxadhocprd.dfs.core.windows.net/audience_factory/powerbi_inputs/eligibility_counts/")

In [0]:
from PIL import Image
import requests
import io
import pandas as pd
import pyspark.sql.functions as f
from pyspark.sql.window import Window
import commodity_segmentations.config as con
import matplotlib.pyplot as plt
import random
import resources.config as config
import seaborn as sns
import os

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def get_image_size(url):
  """ Returns size of the image located at the inputted URL.

  Example
  ----------
    get_image_size('https://www.kroger.com/product/images/medium/front/0019264047717')

  Parameters
  ----------
  url: str
    URL string of where the image is located.

  Returns
  ----------
  A two-dimension tuple with the size image or None if there is no
  image at the inputted URL.
  """
  try:
      # Download the image from the URL
      response = requests.get(url)
      response.raise_for_status()  # Raise an exception for 4xx and 5xx status codes

      # Open the image using Pillow
      image_data = io.BytesIO(response.content)
      image = Image.open(image_data)

      # Get the size of the image
      width, height = image.size

      return (width, height)
  except Exception as e:
      print("Error:", e)
      return None

def draw_nonempty_upcs(df, n=25, upc_colname = 'gtin_no'):
  """Randomly draws UPCs and checks if they have an image
  at https://www.kroger.com/product/images/medium/front/.
  Re-draws the UPC if there is no image found for that UPC.

  Example
  ----------
    draw_nonempty_upcs(df, 25, 'gtin_no')

  Parameters
  ----------
  df: pyspark.sql.dataframe.DataFrame
    PySpark dataframe that has a column with UPC codes.

  n: int
    Integer that defines how many UPCs to draw.

  upc_colname: str
    String that defines the column name in df that contains the
    UPC values. 

  Returns
  ----------
  A list that contains the randomly drawn UPC codes that are confirmed
  to have an image at https://www.kroger.com/product/images/medium/front/.
  """
  #We do a single collect of 50000 randomly drawn UPCs
  #store in a list and cut down on processing time during the re-draws.
  upc_bank = df.\
    select(upc_colname).\
    sample(False, 0.1).\
    limit(50000).\
    rdd.map(lambda x: x[0]).\
    collect()

  #Initially draw of UPCs  
  random_upcs = random.sample(upc_bank, n)
  
  url_fmt = "https://www.kroger.com/product/images/medium/front/{}"
  bad_upcs = []
  good_upcs = []
  dont_redraw = []
  redrawn_upcs = random_upcs
  while len(redrawn_upcs) != 0:
    #For each UPC, check if they have no image
    for upc in redrawn_upcs:
      url = url_fmt.format(upc)
      size = get_image_size(url)

      #If there is no image, then add them to bad UPC list for re-draw
      if size is None:
        print("{} is a bad UPC!".format(upc))
        bad_upcs += [upc]
        dont_redraw += [upc]

      #Is there is an image, then add them to good UPC list for the output
      else:
        good_upcs += [upc]
        dont_redraw += [upc]

    #Drop the UPCs that do not have an image and re-draw from the dataframe
    if len(bad_upcs) != 0:
      print("Executing re-draw...")

      #Filter out the UPCs we have already drawn
      [upc_bank.remove(d) for d in dont_redraw]
      #Define n that we have to re-draw
      redraw_n = len(bad_upcs)
      #Reset bad UPC list for the next iteration
      bad_upcs = []
      #Reset the don't-redraw deck since we already dropped those from the bank
      dont_redraw = []

      #Do the re-draw
      if len(upc_bank) != 0:
        redrawn_upcs = random.sample(upc_bank, redraw_n)

      #Ran out of UPCs. There is nothing to re-draw from.
      else:
        redrawn_upcs = []
        print("Could not redraw any more UPCs. Ran out of UPCs.")
    
    #There is no bad UPCs to re-draw
    else:
      redrawn_upcs = []

  #We tried re-drawing as much as possible - no UPC had an image to show
  if len(good_upcs) == 0:
    raise ValueError("No UPC in the list has an image at www.kroger.com.")
  #We tried re-drawing, but could only get so many images
  elif len(good_upcs) > 0 and len(good_upcs) < n:
    print("Only {} UPCs had an image at www.kroger.com.")
  
  upc_with_url = []
  for i in good_upcs:
    upc = url_fmt.format(i)
    upc_with_url.append(upc)

  return(good_upcs, upc_with_url)

def display_upcs(upcs):
  """Returns an image from https://www.kroger.com/product/images/medium/front/
  for each UPC in the inputted list.

  Example
  ----------
    display_upcs(['0082778285046', '0007248921915', '0081235026102'])

  Parameters
  ----------
  upcs: list
    List of strings where each string is a UPC code. UPC codes
    need to be string since leading zeroes matter.

  Returns
  ----------
  A pandas.io.formats.style.Styler object that has an image
  for each inputted UPC.
  """
  ldf=pd.DataFrame(upcs,columns=['UPC'])
  ldf = ldf.sort_values(by='UPC', ascending=False)
  ldf = ldf.reset_index(drop=True)
  ldf['IMG']=ldf['UPC'].map(lambda x: f'<img src="https://www.kroger.com/product/images/medium/front/{x}">' )#if x.isnumeric() else '')
  return ldf.T.style

def top_words(df, string_col, n=100):
  """Returns the top n words for the inputted column
  in df.

  Example
  ----------
    top_words(df, 'product_name', 100)

  Parameters
  ----------
  df: pyspark.sql.dataframe.DataFrame
    PySpark dataframe that has the string column we'd
    like to extract the top n words from.

  string_col: str
    String that the defines the column name of the 
    column of interest. 

  n: int
    Integer that defines the top n cut-off.

  Returns
  ----------
  A list where the first element is the most frequent word and the
  last element is the n-th most frequent word. Each element is formatted
  as '{word} ({count})' where the parentheses contains the count
  of how many times that word occured when considering all values 
  in the given column. 
  """
  #Get the complete word occurence in each commodity/sub-commodity
  word_count = df.select(f.explode(f.split(df[string_col], " ")).alias("word"))
  word_count = word_count.withColumn("word", f.regexp_replace("word", ",", ""))
  word_count = word_count.withColumn("length", f.length("word"))
  word_count = word_count.filter(f.col("length") > 0)
  word_count = word_count.groupBy(["word"]).count()
  windowSpec = Window.orderBy(f.desc("count"))
  word_count = word_count.withColumn("row_index", f.row_number().over(windowSpec))

  #Collect the top n words and output them as a list since PySpark only
  #shows a maximum of 50 rows
  word_count = word_count.filter(f.col("row_index") <= n)
  words = word_count.select("word").collect()
  words = [row[0] for row in words]
  counts = word_count.select("count").collect()
  counts = [row[0] for row in counts]
  top_words = ["{} ({})".format(w, c) for w, c in zip(words, counts)]
  
  return(top_words)

def display_counts(df, col, limit=15):
  """Returns the grouped-by counts for the inputted
  column and visualizes the counts via a bar chart.

  Example
  ----------
    display_counts(df, "department", 15)

  Parameters
  ----------
  df: pyspark.sql.dataframe.DataFrame
    PySpark dataframe contains the data we'd like
    to conduct the group-by count on.

  col: str
    String that the defines the column name of the 
    column of interest. 

  limit: int
    Integer that defines the the cut-off of how many
    bars to plot in the bar chart. Reduces cluttering.

  Returns
  ----------
  None. Counts and visual are displayed as cell output.
  """
  #Get counts - organize from most dominant to least
  counts_df = df.\
    groupBy(col).\
    count().\
    orderBy('count', ascending=False)
  #Display counts of top 50
  counts_df.show(50, truncate=False)

  #Limit the bar plot to top 15 to avoid overcluttering
  windowSpec = Window.orderBy(f.desc("count"))
  counts_df = counts_df.withColumn("row_index", f.row_number().over(windowSpec))
  counts_df = counts_df.filter(f.col("row_index") <= limit)

  # Convert Spark DataFrame to Pandas DataFrame for plotting
  counts_df = counts_df.toPandas()

  # Plotting
  plt.figure(figsize=(10, 6))
  plt.bar(counts_df[col], counts_df['count'])
  plt.xlabel(col)
  plt.ylabel('Counts')
  plt.title('Counts by {}'.format(col))
  plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
  plt.tight_layout()  # Adjust layout to prevent clipping of labels
  plt.show()

def display_histplot(df, col):
  """Visualizes a histogram for the inputted
  column in the given dataframe.

  Example
  ----------
    display_histplot(df, "dot_product")

  Parameters
  ----------
  df: pyspark.sql.dataframe.DataFrame
    PySpark dataframe contains the data we'd like
    to conduct the group-by count on.

  col: str
    String that the defines the column name of the 
    column of interest.

  Returns
  ----------
  None. Visual is displayed as cell output.
  """

  pandas_df = df.toPandas()

  plt.figure(figsize=(10, 6))
  sns.histplot(pandas_df[col], kde=True)
  plt.title("Distribution of {}".format(col))
  plt.xlabel("Values")
  plt.ylabel("Frequency")
  plt.show()

def write_out(df, fp, delim=",", fmt="csv"):
  """Writes out PySpark dataframe as a csv file
  that can be downloaded for Azure and loaded into
  Excel very easily.

  Example
  ----------
    write_out(df, "abfss://media@sa8451dbxadhocprd.dfs.core.windows.net/audience_factory/adhoc/data_analysis.csv")

  Parameters
  ----------
  df: pyspark.sql.dataframe.DataFrame
    PySpark dataframe contains the data we'd like
    to conduct the group-by count on.

  fp: str
    String that the defines the column name of the 
    column of interest.

  delim: str
    String that specifies which delimiter to use in the
    write-out. Default value is ','.

  fmt: str
    String that specifies which format to use in the
    write-out. Default value is 'csv'.

  Returns
  ----------
  None. File is written out specified Azure location.
  """
  #Placeholder filepath for intermediate processing
  temp_target = os.path.dirname(fp) + "/" + "temp"

  #Write out df as partitioned file. Write out ^ delimited
  df.coalesce(1).write.options(header=True, delimiter=delim).mode("overwrite").format(fmt).save(temp_target)

  #Copy desired file from parititioned directory to desired location
  temporary_fp = os.path.join(temp_target, dbutils.fs.ls(temp_target)[3][1])
  dbutils.fs.cp(temporary_fp, fp)
  dbutils.fs.rm(temp_target, recurse=True)


In [0]:
import subprocess
from functools import reduce
from effodata import ACDS, golden_rules, Joiner, Sifter, Equality, join_on
from kpi_metrics import KPI, AliasMetric, CustomMetric, AliasGroupby
from IPython.display import FileLink 
import sys
from typing import Union
import datetime as dt
from datetime import timedelta, datetime, date

segmentation_list = segmentation_name
acds = ACDS(use_sample_mart = False)

In [0]:
srt_dt = (date.today() - timedelta(days=385)).strftime("%Y%m%d")
end_dt = (date.today() - timedelta(days=10)).strftime("%Y%m%d")
srt_dt, end_dt

In [0]:

my_schema_elig = t.StructType([
    t.StructField("UPC", t.StringType(), True),
    t.StructField("URL", t.StringType(), True),
    t.StructField("SEGMENTATION", t.StringType(), True),
])
final_df = spark.createDataFrame([], schema=my_schema_elig)
failed_seg = {}

for i in segmentation_list: 
    try: 
      seg_dir = f"{con.output_fp}upc_lists/{i}/"
      dir_contents = dbutils.fs.ls(seg_dir)
      dir_contents = [x[0] for x in dir_contents if i in x[1]]
      dir_contents.sort()
      seg_fp = dir_contents[-1]
      df = spark.read.format("delta").load(seg_fp)
      transaction = acds.get_transactions(start_date=srt_dt, end_date=end_dt)
      gtins = (transaction.groupBy('gtin_no')
              .agg(f.count('transaction_code').alias('Total Redemers'),
                  f.count('ehhn').alias("Total Households"),
                  f.countDistinct('ehhn').alias("Total Distinct Counts"))
              .sort('Total Redemers', ascending=False))
      getting_joined = df.join(gtins, on='gtin_no', how='left').sort('Total Redemers', ascending=False).select('gtin_no')
      n = 10
      image_upcs, upc_url = draw_nonempty_upcs(getting_joined, n)

      # Combine the lists using zip
      combined_data = zip(image_upcs, upc_url)

      # Create a DataFrame with column names
      df = spark.createDataFrame(combined_data, ["UPC", "URL"])
      upc_file = (df.withColumn("SEGMENTATION", f.lit(i)))

      final_df = final_df.union(upc_file)

    except Exception as e:
          failed_seg[i] = str(e)       
      




In [0]:
final_df.coalesce(1).write.mode("overwrite").parquet(f"abfss://media@sa8451dbxadhocprd.dfs.core.windows.net/audience_factory/powerbi_inputs/upc_via_segmentation/")

In [0]:
failed_seg.keys()